<a href="https://colab.research.google.com/github/weixinluo524/AAI2026/blob/main/Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prompt #1 I used in CHATGPT: can you help me build a simple clothing store customer service chatbot in python, it should have basic store hours, emails, shipping and return function. It should run smoothly.




In [30]:
#!/usr/bin/env python3
"""
Simple Clothing Store Customer Service Chatbot (Terminal)
- Rule-based intent detection (fast, reliable, no dependencies)
- Session memory (name, last order)
- Common support flows: order status, returns, shipping, sizing, product availability, hours, contact
"""

from __future__ import annotations
import re
import sys
from dataclasses import dataclass, field
from datetime import datetime
from typing import Dict, List, Optional, Tuple


# ---------- Config you can edit ----------
STORE = {
    "name": "MEWO MEOW Shop",
    "hours": "Mon–Fri 9am–6pm, Sat 10am–4pm, Sun closed (local time)",
    "support_email": "support@threadedlane.example",
    "support_phone": "+1 (555) 010-2020",
    "shipping": {
        "standard": "3–5 business days",
        "express": "1–2 business days",
        "free_over": 75.00,
    },
    "returns": {
        "window_days": 30,
        "condition": "unworn, unwashed, with tags attached",
        "refund_timing": "3–7 business days after we receive your return",
    },
    "size_guide": {
        "tops": "If between sizes, size up for a relaxed fit; check chest measurement.",
        "bottoms": "Measure waist at natural waist; if between sizes, choose based on fit preference.",
        "shoes": "If between sizes, size up; wide feet may prefer half-size up.",
    },
}

# Mock “database” so the bot can answer without integrations.
MOCK_ORDERS = {
    # order_id: (status, eta, carrier)
    "TL10001": ("Shipped", "Mar 2, 2026", "UPS"),
    "TL10002": ("Processing", "Mar 5, 2026", "USPS"),
    "TL10003": ("Delivered", "Feb 24, 2026", "FedEx"),
}

MOCK_INVENTORY = {
    # product keywords -> in stock sizes/colors
    "hoodie": {"sizes": ["XS", "S", "M", "L", "XL"], "colors": ["black", "heather gray", "navy"]},
    "jeans": {"sizes": ["28", "30", "32", "34", "36"], "colors": ["indigo", "black"]},
    "tshirt": {"sizes": ["XS", "S", "M", "L", "XL", "XXL"], "colors": ["white", "black", "sand"]},
    "dress": {"sizes": ["0", "2", "4", "6", "8", "10"], "colors": ["emerald", "black"]},
}


# ---------- Session / State ----------
@dataclass
class Session:
    customer_name: Optional[str] = None
    last_order_id: Optional[str] = None
    context: Dict[str, str] = field(default_factory=dict)


# ---------- Intent detection ----------
INTENTS = [
    "greeting",
    "goodbye",
    "help",
    "order_status",
    "returns",
    "exchange",
    "shipping",
    "sizing",
    "availability",
    "hours",
    "contact",
    "agent",
    "unknown",
]

ORDER_ID_RE = re.compile(r"\b(?:TL)?\d{5}\b", re.IGNORECASE)

def normalize(text: str) -> str:
    return re.sub(r"\s+", " ", text.strip().lower())

def extract_order_id(text: str) -> Optional[str]:
    m = ORDER_ID_RE.search(text)
    if not m:
        return None
    oid = m.group(0).upper()
    if not oid.startswith("TL"):
        oid = "TL" + oid
    return oid

def detect_intent(user_text: str) -> str:
    t = normalize(user_text)

    # greetings / goodbyes
    if any(x in t for x in ["hi", "hello", "hey", "good morning", "good afternoon", "good evening"]):
        return "greeting"
    if any(x in t for x in ["bye", "goodbye", "see you", "thanks", "thank you", "thx"]):
        # "thanks" is not necessarily goodbye, but keeps terminal bots pleasant
        if t in ["thanks", "thank you", "thx", "thanks!"]:
            return "help"
        return "goodbye"

    # explicit help
    if any(x in t for x in ["help", "what can you do", "options", "menu"]):
        return "help"

    # routing by keywords
    if any(x in t for x in ["order", "tracking", "track", "status", "where is my"]):
        return "order_status"
    if any(x in t for x in ["return", "refund"]):
        return "returns"
    if any(x in t for x in ["exchange", "swap", "different size"]):
        return "exchange"
    if any(x in t for x in ["shipping", "delivery", "deliver", "ship"]):
        return "shipping"
    if any(x in t for x in ["size", "sizing", "fit", "measurements"]):
        return "sizing"
    if any(x in t for x in ["in stock", "available", "availability", "do you have"]):
        return "availability"
    if any(x in t for x in ["hours", "open", "closing", "when are you open"]):
        return "hours"
    if any(x in t for x in ["contact", "email", "phone", "support"]):
        return "contact"
    if any(x in t for x in ["human", "agent", "representative", "someone", "person"]):
        return "agent"

    return "unknown"


# ---------- Response helpers ----------
def bot_prefix() -> str:
    return f"{STORE['name']} Bot"

def format_money(x: float) -> str:
    return f"${x:,.2f}"

def render_help() -> str:
    return (
        "I can help with:\n"
        "• Order status / tracking (e.g., “track TL10001”)\n"
        "• Returns & refunds (return window, steps)\n"
        "• Exchanges (size/color swaps)\n"
        "• Shipping options & costs\n"
        "• Sizing guidance\n"
        "• Product availability\n"
        "• Store hours & contact info\n"
        "Type 'quit' to exit."
    )

def ask_name_if_missing(session: Session) -> Optional[str]:
    if session.customer_name:
        return None
    return "Before we start—what name should I use?"

def friendly_name(session: Session) -> str:
    return session.customer_name or "there"

def safe_fallback() -> str:
    return (
        "I’m not totally sure I got that. You can say things like:\n"
        "• “track TL10001”\n"
        "• “return policy”\n"
        "• “what are your shipping options?”\n"
        "• “size help for jeans”"
    )


# ---------- Handlers ----------
def handle_greeting(session: Session) -> str:
    return f"Hi {friendly_name(session)}! 👋 How can I help you today? (Try: order status, returns, shipping, sizing.)"

def handle_order_status(session: Session, user_text: str) -> str:
    oid = extract_order_id(user_text) or session.last_order_id
    if not oid:
        return "Sure — what’s your order number? (Example: TL10001)"
    session.last_order_id = oid

    info = MOCK_ORDERS.get(oid)
    if not info:
        return (
            f"I couldn’t find order {oid} in my demo system. "
            "Double-check the number, or say “agent” to contact support."
        )

    status, eta, carrier = info
    return f"Order {oid}: **{status}** via {carrier}. Estimated delivery: {eta}."

def handle_returns(session: Session, user_text: str) -> str:
    r = STORE["returns"]
    return (
        f"Returns are accepted within **{r['window_days']} days** of delivery, as long as items are **{r['condition']}**.\n"
        "To start a return:\n"
        "1) Share your order number (e.g., TL10001)\n"
        "2) Tell me which item(s) you’re returning and the reason\n"
        f"Refunds usually process in **{r['refund_timing']}**."
    )

def handle_exchange(session: Session, user_text: str) -> str:
    oid = extract_order_id(user_text) or session.last_order_id
    if not oid:
        return "Happy to help with an exchange. What’s your order number? (Example: TL10001)"
    session.last_order_id = oid
    return (
        f"Got it. For order {oid}, what would you like to exchange?\n"
        "Reply with:\n"
        "• Item name\n"
        "• Current size/color\n"
        "• Desired size/color"
    )

def handle_shipping(session: Session) -> str:
    s = STORE["shipping"]
    return (
        "Shipping options:\n"
        f"• Standard: {s['standard']}\n"
        f"• Express: {s['express']}\n"
        f"• Free standard shipping on orders over {format_money(s['free_over'])}\n"
        "If you share your order total + destination ZIP (or country), I can estimate the best option."
    )

def handle_sizing(session: Session, user_text: str) -> str:
    t = normalize(user_text)
    guide = STORE["size_guide"]
    if any(x in t for x in ["jean", "pants", "bottom"]):
        return f"Bottoms sizing tip: {guide['bottoms']}"
    if any(x in t for x in ["shoe", "sneaker", "boot"]):
        return f"Shoe sizing tip: {guide['shoes']}"
    if any(x in t for x in ["shirt", "tee", "top", "hoodie", "sweater"]):
        return f"Tops sizing tip: {guide['tops']}"
    return (
        "What are you shopping for — tops, bottoms, or shoes? "
        "If you share your usual size and fit preference (snug/regular/relaxed), I’ll recommend a size."
    )

def handle_availability(session: Session, user_text: str) -> str:
    t = normalize(user_text)
    matches: List[Tuple[str, Dict[str, List[str]]]] = []
    for key, info in MOCK_INVENTORY.items():
        if key in t:
            matches.append((key, info))

    if not matches:
        return (
            "Which item are you checking? (Examples: hoodie, jeans, tshirt, dress)\n"
            "You can also include preferred size/color."
        )

    lines = []
    for key, info in matches:
        sizes = ", ".join(info["sizes"])
        colors = ", ".join(info["colors"])
        lines.append(f"{key.title()}: sizes [{sizes}] • colors [{colors}]")
    return "Here’s what I have in the demo inventory:\n" + "\n".join(lines)

def handle_hours() -> str:
    return f"Our hours are: **{STORE['hours']}**."

def handle_contact() -> str:
    return (
        "You can reach support here:\n"
        f"• Email: {STORE['support_email']}\n"
        f"• Phone: {STORE['support_phone']}\n"
        "If you tell me your issue + order number, I can format a message to send."
    )

def handle_agent() -> str:
    return (
        "No problem — I can’t transfer you directly in this demo, but here are the fastest ways to reach a human:\n"
        f"• {STORE['support_email']}\n"
        f"• {STORE['support_phone']}\n"
        "If you paste what happened (and your order number), I’ll draft a short support email."
    )

def handle_unknown() -> str:
    return safe_fallback()


# ---------- Main loop ----------
def route(session: Session, user_text: str) -> str:
    intent = detect_intent(user_text)

    # capture simple name: "my name is X"
    t = user_text.strip()
    m = re.search(r"\bmy name is\s+([A-Za-z][A-Za-z-']{0,30})\b", t, re.IGNORECASE)
    if m:
        session.customer_name = m.group(1)
        return f"Nice to meet you, {session.customer_name}! How can I help?"

    # ask name early but not blocking
    name_prompt = ask_name_if_missing(session)
    if name_prompt and intent not in ["greeting", "help", "goodbye"]:
        # We'll still answer, but nudge once.
        session.context["asked_name"] = "1"

    if intent == "greeting":
        return handle_greeting(session)
    if intent == "help":
        extra = ""
        if name_prompt and not session.context.get("asked_name"):
            session.context["asked_name"] = "1"
            extra = f"\n\n{ name_prompt } (Or say “skip”.)"
        return render_help() + extra
    if intent == "order_status":
        return handle_order_status(session, user_text)

    if intent == "returns":

        return handle_returns(session, user_text)
    if intent == "exchange":
        return handle_exchange(session, user_text)
    if intent == "shipping":
        return handle_shipping(session)
    if intent == "sizing":
        return handle_sizing(session, user_text)
    if intent == "availability":
        return handle_availability(session, user_text)
    if intent == "hours":
        return handle_hours()
    if intent == "contact":
        return handle_contact()
    if intent == "agent":
        return handle_agent()
    if intent == "goodbye":
        return f"Thanks, {friendly_name(session)} — take care! 👋"
    return handle_unknown()


def main() -> int:
    session = Session()
    print(f"{bot_prefix()}: Hello! I’m your customer service assistant. Type 'help' for options.\n")

    while True:
        try:
            user_text = input("You: ").strip()
        except (EOFError, KeyboardInterrupt):
            print(f"\n{bot_prefix()}: Bye! 👋")
            return 0

        if not user_text:
            print(f"{bot_prefix()}: {safe_fallback()}")
            continue

        if normalize(user_text) in ["quit", "exit", "q"]:
            print(f"{bot_prefix()}: Bye! 👋")
            return 0

        if normalize(user_text) == "skip" and not session.customer_name:
            session.customer_name = "friend"
            print(f"{bot_prefix()}: No worries — how can I help?")
            continue

        response = route(session, user_text)
        print(f"{bot_prefix()}: {response}")

    # unreachable
    # return 0


if __name__ == "__main__":
    main()

MEWO MEOW Shop Bot: Hello! I’m your customer service assistant. Type 'help' for options.

You: help
MEWO MEOW Shop Bot: I can help with:
• Order status / tracking (e.g., “track TL10001”)
• Returns & refunds (return window, steps)
• Exchanges (size/color swaps)
• Shipping options & costs
• Sizing guidance
• Product availability
• Store hours & contact info
Type 'quit' to exit.

Before we start—what name should I use? (Or say “skip”.)
You: my name is weixin
MEWO MEOW Shop Bot: Nice to meet you, weixin! How can I help?
You: return
MEWO MEOW Shop Bot: Returns are accepted within **30 days** of delivery, as long as items are **unworn, unwashed, with tags attached**.
To start a return:
1) Share your order number (e.g., TL10001)
2) Tell me which item(s) you’re returning and the reason
Refunds usually process in **3–7 business days after we receive your return**.
You: TL10021
MEWO MEOW Shop Bot: I’m not totally sure I got that. You can say things like:
• “track TL10001”
• “return policy”
• “wh

Prompt #2 Code Generation with ReACT Prompting with CHATGPT: Imagine you are a IT role in a company, your boss wants you to create a simple chatbot that could simulate to log sick leaves for every week. The employee can communicate with bot to schedule for sick leave. It should be simple and can do a small simulation in google colab. The code needs to *be* executable and in python


In [23]:
import re
from datetime import date, datetime, timedelta

# -------------------- Helpers --------------------
DAYS = ["mon", "tue", "wed", "thu", "fri", "sat", "sun"]
FULLDAY = {
    "monday":"mon","tuesday":"tue","wednesday":"wed","thursday":"thu",
    "friday":"fri","saturday":"sat","sunday":"sun"
}

def iso_week_str(d: date) -> str:
    y, w, _ = d.isocalendar()
    return f"{y}-W{w:02d}"

def parse_week(text: str, base: date) -> str:
    t = text.lower()
    if "this week" in t:
        return iso_week_str(base)
    if "next week" in t:
        return iso_week_str(base + timedelta(days=7))
    m = re.search(r"(\d{4})-w(\d{1,2})", t)
    if m:
        y = int(m.group(1)); w = int(m.group(2))
        if 1 <= w <= 53:
            return f"{y}-W{w:02d}"
    m2 = re.search(r"\bweek\s*(\d{1,2})\b", t)
    if m2:
        w = int(m2.group(1))
        if 1 <= w <= 53:
            return f"{base.year}-W{w:02d}"
    return ""

def parse_day(text: str, base: date) -> str:
    t = text.lower()
    if "today" in t:
        return DAYS[base.weekday()]
    if "tomorrow" in t:
        return DAYS[(base.weekday() + 1) % 7]
    for k, v in FULLDAY.items():
        if k in t:
            return v
    for d in DAYS:
        if re.search(rf"\b{d}\b", t):
            return d
    return ""

def parse_reason(text: str) -> str:
    m = re.search(r"\bbecause\b(.+)$", text, flags=re.IGNORECASE)
    if m:
        return m.group(1).strip().strip(".")
    m2 = re.search(r"\breason\s*:\s*(.+)$", text, flags=re.IGNORECASE)
    if m2:
        return m2.group(1).strip().strip(".")
    return "Not specified"

def pretty_day(d: str) -> str:
    return {"mon":"Mon","tue":"Tue","wed":"Wed","thu":"Thu","fri":"Fri","sat":"Sat","sun":"Sun"}.get(d, d)

def help_text() -> str:
    return (
        "Commands you can try:\n"
        "- log sick leave mon this week because flu\n"
        "- log sick leave today because fever\n"
        "- log sick leave fri 2026-W09 because migraine\n"
        "- show\n"
        "- cancel mon this week\n"
        "- help\n"
        "- quit"
    )

# -------------------- In-memory DB --------------------
# leave_db = { user: { week: { day: {reason, logged_at} } } }
leave_db = {}

def ensure_user(u: str):
    leave_db.setdefault(u, {})

def log_leave(u: str, week: str, day: str, reason: str):
    ensure_user(u)
    leave_db[u].setdefault(week, {})
    leave_db[u][week][day] = {
        "reason": reason,
        "logged_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

def cancel_leave(u: str, week: str, day: str) -> bool:
    if u in leave_db and week in leave_db[u] and day in leave_db[u][week]:
        del leave_db[u][week][day]
        if not leave_db[u][week]:
            del leave_db[u][week]
        return True
    return False

def show_leaves(u: str) -> str:
    ensure_user(u)
    if not leave_db[u]:
        return "No sick leave entries found."
    lines = []
    for week in sorted(leave_db[u].keys()):
        lines.append(f"{week}:")
        for d in DAYS:
            if d in leave_db[u][week]:
                e = leave_db[u][week][d]
                lines.append(f"  - {pretty_day(d)}: SICK | reason: {e['reason']} | logged: {e['logged_at']}")
        lines.append("")
    return "\n".join(lines).strip()

# -------------------- "Chatty" transcript UI --------------------
history = []

def say(role: str, text: str):
    ts = datetime.now().strftime("%H:%M:%S")
    history.append((role, text, ts))

    # Re-print transcript (chat-like)
    print("\n" + "=" * 60)
    for r, t, time_str in history[-40:]:
        who = "You" if r == "user" else "Bot"
        print(f"[{time_str}] {who}: {t}")
    print("=" * 60 + "\n")

def handle_message(user: str, msg: str, base: date):
    low = msg.lower().strip()

    if low in ("quit", "exit", "bye"):
        return None

    if low in ("help", "?"):
        return help_text()

    if low.startswith("show"):
        return show_leaves(user)

    if low.startswith("cancel"):
        week = parse_week(msg, base) or iso_week_str(base)
        day = parse_day(msg, base)
        if not day:
            return "Which day to cancel? Example: cancel mon this week"
        ok = cancel_leave(user, week, day)
        return (f"Cancelled sick leave for {pretty_day(day)} in {week}."
                if ok else f"No entry found for {pretty_day(day)} in {week}.")

    if ("log" in low) and ("sick" in low):
        week = parse_week(msg, base) or iso_week_str(base)
        day = parse_day(msg, base)
        if not day:
            return "Which day? Example: log sick leave mon this week because <reason>"
        reason = parse_reason(msg)
        log_leave(user, week, day, reason)
        return f"Logged sick leave for {pretty_day(day)} in {week}. Reason: {reason}"

    return "I didn't understand. Type 'help'."

# -------------------- Run chatbot --------------------
def run_chatbot():
    base = date.today()
    user = input("Employee username (e.g., alice): ").strip() or "employee"
    ensure_user(user)

    say("bot", f"Hi {user}! I can log sick leave. Type 'help' for commands.")
    say("bot", f"Today is {base.isoformat()} (ISO week: {iso_week_str(base)}).")

    while True:
        msg = input("You: ").strip()
        if not msg:
            continue
        say("user", msg)

        reply = handle_message(user, msg, base)
        if reply is None:
            say("bot", "Goodbye! 👋")
            break
        say("bot", reply)

run_chatbot()

Employee username (e.g., alice): weixin

[18:28:45] Bot: Hi weixin! I can log sick leave. Type 'help' for commands.


[18:28:45] Bot: Hi weixin! I can log sick leave. Type 'help' for commands.
[18:28:45] Bot: Today is 2026-02-28 (ISO week: 2026-W09).

You: help

[18:28:45] Bot: Hi weixin! I can log sick leave. Type 'help' for commands.
[18:28:45] Bot: Today is 2026-02-28 (ISO week: 2026-W09).
[18:28:48] You: help


[18:28:45] Bot: Hi weixin! I can log sick leave. Type 'help' for commands.
[18:28:45] Bot: Today is 2026-02-28 (ISO week: 2026-W09).
[18:28:48] You: help
[18:28:48] Bot: Commands you can try:
- log sick leave mon this week because flu
- log sick leave today because fever
- log sick leave fri 2026-W09 because migraine
- show
- cancel mon this week
- help
- quit

You: log sick leave today as flu and stomach hurt

[18:28:45] Bot: Hi weixin! I can log sick leave. Type 'help' for commands.
[18:28:45] Bot: Today is 2026-02-28 (ISO week: 2026-W09).
[18:28:48] You: help
[18:28:48] B

Prompt 3 Self-Reflection: For the code you generated, when I asked to log sick leave in a different reason, it wont proceed or will show reason:not specified. The reasons for sick leaves should be anything.  please revise on that.



In [22]:
import re
from datetime import date, datetime, timedelta

# -------------------- Helpers --------------------
DAYS = ["mon", "tue", "wed", "thu", "fri", "sat", "sun"]
FULLDAY = {
    "monday":"mon","tuesday":"tue","wednesday":"wed","thursday":"thu",
    "friday":"fri","saturday":"sat","sunday":"sun"
}

def iso_week_str(d: date) -> str:
    y, w, _ = d.isocalendar()
    return f"{y}-W{w:02d}"

def parse_week(text: str, base: date) -> str:
    t = text.lower()
    if "this week" in t:
        return iso_week_str(base)
    if "next week" in t:
        return iso_week_str(base + timedelta(days=7))
    m = re.search(r"(\d{4})-w(\d{1,2})", t)
    if m:
        y = int(m.group(1)); w = int(m.group(2))
        if 1 <= w <= 53:
            return f"{y}-W{w:02d}"
    m2 = re.search(r"\bweek\s*(\d{1,2})\b", t)
    if m2:
        w = int(m2.group(1))
        if 1 <= w <= 53:
            return f"{base.year}-W{w:02d}"
    return ""

def parse_day(text: str, base: date) -> str:
    t = text.lower()
    if "today" in t:
        return DAYS[base.weekday()]
    if "tomorrow" in t:
        return DAYS[(base.weekday() + 1) % 7]
    for k, v in FULLDAY.items():
        if k in t:
            return v
    for d in DAYS:
        if re.search(rf"\b{d}\b", t):
            return d
    return ""

def pretty_day(d: str) -> str:
    return {"mon":"Mon","tue":"Tue","wed":"Wed","thu":"Thu","fri":"Fri","sat":"Sat","sun":"Sun"}.get(d, d)

def help_text() -> str:
    return (
        "Commands you can try:\n"
        "- log sick leave mon this week flu and fever\n"
        "- log sick leave today migraine\n"
        "- log sick leave fri 2026-W09 stomach bug\n"
        "- log sick leave mon this week because flu\n"
        "- show\n"
        "- cancel mon this week\n"
        "- help\n"
        "- quit"
    )

def extract_reason_freeform(msg: str, base: date) -> str:
    """
    Accept ANY reason text, no required pattern.

    Priority:
    1) If message has 'because ...' or 'reason: ...', use that tail.
    2) Otherwise, remove common command tokens/day/week tokens and treat the rest as reason.
    """
    raw = msg.strip()

    # Pattern-based (optional, but supported)
    m = re.search(r"\bbecause\b(.+)$", raw, flags=re.IGNORECASE)
    if m:
        return m.group(1).strip().strip(" .")
    m2 = re.search(r"\breason\s*:\s*(.+)$", raw, flags=re.IGNORECASE)
    if m2:
        return m2.group(1).strip().strip(" .")

    # Freeform fallback
    t = raw.lower()

    # Remove leading command phrase (flexible)
    t = re.sub(r"^\s*log\s+sick(\s+leave)?\s*", "", t, flags=re.IGNORECASE)

    # Remove common filler words
    t = re.sub(r"\b(on|for|in|at)\b", " ", t)

    # Remove week expressions
    t = re.sub(r"\bthis\s+week\b", " ", t)
    t = re.sub(r"\bnext\s+week\b", " ", t)
    t = re.sub(r"\bweek\s*\d{1,2}\b", " ", t)
    t = re.sub(r"\b\d{4}-w\d{1,2}\b", " ", t)

    # Remove day expressions
    t = re.sub(r"\btoday\b", " ", t)
    t = re.sub(r"\btomorrow\b", " ", t)
    for k in FULLDAY.keys():
        t = re.sub(rf"\b{k}\b", " ", t)
    for d in DAYS:
        t = re.sub(rf"\b{d}\b", " ", t)

    # Clean up spaces
    reason = re.sub(r"\s+", " ", t).strip(" .-")

    return reason if reason else "Not specified"

# -------------------- In-memory DB --------------------
# leave_db = { user: { week: { day: {reason, logged_at} } } }
leave_db = {}

def ensure_user(u: str):
    leave_db.setdefault(u, {})

def log_leave(u: str, week: str, day: str, reason: str):
    ensure_user(u)
    leave_db[u].setdefault(week, {})
    leave_db[u][week][day] = {
        "reason": reason,
        "logged_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

def cancel_leave(u: str, week: str, day: str) -> bool:
    if u in leave_db and week in leave_db[u] and day in leave_db[u][week]:
        del leave_db[u][week][day]
        if not leave_db[u][week]:
            del leave_db[u][week]
        return True
    return False

def show_leaves(u: str) -> str:
    ensure_user(u)
    if not leave_db[u]:
        return "No sick leave entries found."
    lines = []
    for week in sorted(leave_db[u].keys()):
        lines.append(f"{week}:")
        for d in DAYS:
            if d in leave_db[u][week]:
                e = leave_db[u][week][d]
                lines.append(f"  - {pretty_day(d)}: SICK | reason: {e['reason']} | logged: {e['logged_at']}")
        lines.append("")
    return "\n".join(lines).strip()

# -------------------- "Chatty" transcript UI --------------------
history = []

def say(role: str, text: str):
    ts = datetime.now().strftime("%H:%M:%S")
    history.append((role, text, ts))

    print("\n" + "=" * 60)
    for r, t, time_str in history[-40:]:
        who = "You" if r == "user" else "Bot"
        print(f"[{time_str}] {who}: {t}")
    print("=" * 60 + "\n")

def handle_message(user: str, msg: str, base: date):
    low = msg.lower().strip()

    if low in ("quit", "exit", "bye"):
        return None

    if low in ("help", "?"):
        return help_text()

    if low.startswith("show"):
        return show_leaves(user)

    if low.startswith("cancel"):
        week = parse_week(msg, base) or iso_week_str(base)
        day = parse_day(msg, base)
        if not day:
            return "Which day to cancel? Example: cancel mon this week"
        ok = cancel_leave(user, week, day)
        return (f"Cancelled sick leave for {pretty_day(day)} in {week}."
                if ok else f"No entry found for {pretty_day(day)} in {week}.")

    # Logging: keep it flexible but still specific enough to avoid false triggers
    if "log" in low and "sick" in low:
        week = parse_week(msg, base) or iso_week_str(base)
        day = parse_day(msg, base)
        if not day:
            return "Which day? Example: log sick leave mon this week flu and fever"
        reason = extract_reason_freeform(msg, base)
        log_leave(user, week, day, reason)
        return f"Logged sick leave for {pretty_day(day)} in {week}. Reason: {reason}"

    return "I didn't understand. Type 'help'."

# -------------------- Run chatbot --------------------
def run_chatbot():
    base = date.today()
    user = input("Employee username (e.g., alice): ").strip() or "employee"
    ensure_user(user)

    say("bot", f"Hi {user}! I can log sick leave. Type 'help' for commands.")
    say("bot", f"Today is {base.isoformat()} (ISO week: {iso_week_str(base)}).")

    while True:
        msg = input("You: ").strip()
        if not msg:
            continue
        say("user", msg)

        reply = handle_message(user, msg, base)
        if reply is None:
            say("bot", "Goodbye! 👋")
            break
        say("bot", reply)

run_chatbot()

Employee username (e.g., alice): weixin

[18:28:09] Bot: Hi weixin! I can log sick leave. Type 'help' for commands.


[18:28:09] Bot: Hi weixin! I can log sick leave. Type 'help' for commands.
[18:28:09] Bot: Today is 2026-02-28 (ISO week: 2026-W09).

You: help

[18:28:09] Bot: Hi weixin! I can log sick leave. Type 'help' for commands.
[18:28:09] Bot: Today is 2026-02-28 (ISO week: 2026-W09).
[18:28:13] You: help


[18:28:09] Bot: Hi weixin! I can log sick leave. Type 'help' for commands.
[18:28:09] Bot: Today is 2026-02-28 (ISO week: 2026-W09).
[18:28:13] You: help
[18:28:13] Bot: Commands you can try:
- log sick leave mon this week flu and fever
- log sick leave today migraine
- log sick leave fri 2026-W09 stomach bug
- log sick leave mon this week because flu
- show
- cancel mon this week
- help
- quit

You: log sick leave today as flu and stomach hurt

[18:28:09] Bot: Hi weixin! I can log sick leave. Type 'help' for commands.
[18:28:09] Bot: Today is 2026-02-28 (ISO week: 2026-W09)